In [1]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25861 sha256=2c2ef47edc891ba3e2343f91ae88098abddc2d7381e1c1f9ea793e27ffd768d6
  Stored in directory: /root/.cache/pip/wheels/35/e4/69/cb99d996d14a2971b79b990d68b05a17d58ce530ff96090dfc
Successfully built imutils


In [2]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense, Input, Lambda, Input, Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D
import tensorflow as tf 
import tensorflow.keras as keras 

In [3]:
from imutils import paths 

train_image_files = list(paths.list_images('train'))
test_image_files = list(paths.list_images('test'))
val_image_files = list(paths.list_images('val'))

In [ ]:
from tensorflow import keras 
import tensorflow as tf 

# Image preprocessing utils
def parse_images(image_path): 
    """
        this function, will read the image from the directory, and resize it to the 224 image shape, which is predefined 
        image shape of Resnet pretrained model.
        Params:
            image_path(type: str): image filepath in the directory.
        Return(type: tf.Image)
            returns the preprocessed image.
    """
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[224, 224])

    return image

def create_tensorflow_dataset(data, batch_size):
    """
        this function, will create the tensorflow dataset from the images.
        Params:
            data(type: List): list of data, in our case list contains the file path.
            batch_size(type: Int): batch size for the generator.
        Return(type: tf.data.Dataset)
            returns tensorflow dataset object.
    """
    tensorflow_data = tf.data.Dataset.from_tensor_slices(data)
    tensorflow_data = (
    tensorflow_data
        .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .shuffle(1024)
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return tensorflow_data        

In [19]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

def save_loss_plot(fpath, history): 
    with plt.xkcd():
        plt.plot(history.history["loss"], label="train_loss")
        plt.plot(history.history["val_loss"], label="val_loss")

        plt.title("Training Loss and val loss")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss")
        plt.legend(loc="lower left")
    plt.savefig(fpath)
    plt.close()
    

def save_acc_plot(fpath, history): 
    with plt.xkcd():
        plt.plot(history.history["accuracy"], label="train_acc")
        plt.plot(history.history["val_accuracy"], label="val_acc")
        plt.title("Training and val Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Accuracy")
        plt.legend(loc="lower left")
    plt.savefig(fpath)
    plt.close()

    
def create_dataset(images_files): 
    X = []
    y = []
    for image_file in images_files:
        try: 
            img = cv2.imread(image_file)
            img = cv2.resize(img, (224, 224))
            img = img / 255.0 

            label = image_file.split("/")[1].split("_")[0]

            X.append(img)
            y.append(label)
        
        except Exception as error:
            return error
    
    return np.array(X), np.array(y)

In [5]:
train_X, train_y = create_dataset(train_image_files)
train_X.shape

(2700, 224, 224, 3)

In [6]:
np.unique(train_y)

array(['airplane', 'car', 'elephant', 'lion', 'mouse', 'tiger cub'],
      dtype='<U9')

In [7]:
val_X, val_y = create_dataset(val_image_files)
val_X.shape

(292, 224, 224, 3)

In [8]:
test_X, test_y = create_dataset(test_image_files)
test_X.shape

(300, 224, 224, 3)

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
val_y = label_encoder.transform(val_y)
test_y = label_encoder.transform(test_y)

In [10]:
import tensorflow.keras as keras 
from tensorflow import keras 

base_encoder_model = keras.models.load_model("simclr_model_1")

class ClassificationHed(keras.models.Model): 
    def __init__(self, out_dims, activation_type): 
        super(ClassificationHed, self).__init__()
        self.out_dims = out_dims 
        self.activation_type = activation_type
        
        self.out_layer = keras.layers.Dense(self.out_dims, activation=self.activation_type)
    
    def call(self, tensor): 
        x = self.out_layer(tensor)
        return x 


In [11]:
encoder_non_linearity_model = keras.models.Sequential([
    base_encoder_model.layers[0],
    base_encoder_model.layers[1],
    ClassificationHed(6, 'softmax')
])

for layer in encoder_non_linearity_model.layers[:-1]: 
    layer.trainable = False
    
encoder_non_linearity_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_3 (Functional)        (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 classification_hed (Classif  (None, 6)                12294     
 icationHed)                                                     
                                                                 
Total params: 23,600,006
Trainable params: 12,294
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
encoder_non_linearity_model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"],
                                                     optimizer='adam')

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',
                                    patience=5,
                                    verbose=0,
                                    mode='auto',
                                    baseline=None,
                                    restore_best_weights=True,
                                )
history = encoder_non_linearity_model.fit(train_X, train_y, batch_size=64, epochs=30, validation_data=(val_X, val_y))

Epoch 1/30
43/43 [==============================] - 16s 168ms/step - loss: 2.5646 - accuracy: 0.2204 - val_loss: 1.9056 - val_accuracy: 0.2055
Epoch 2/30
43/43 [==============================] - 5s 127ms/step - loss: 1.8255 - accuracy: 0.2785 - val_loss: 1.9274 - val_accuracy: 0.3014
Epoch 3/30
43/43 [==============================] - 6s 132ms/step - loss: 1.8591 - accuracy: 0.2770 - val_loss: 2.0465 - val_accuracy: 0.3151
Epoch 4/30
43/43 [==============================] - 5s 122ms/step - loss: 1.8100 - accuracy: 0.2985 - val_loss: 1.8834 - val_accuracy: 0.2568
Epoch 5/30
43/43 [==============================] - 5s 122ms/step - loss: 1.8342 - accuracy: 0.2941 - val_loss: 2.0320 - val_accuracy: 0.3390
Epoch 6/30
43/43 [==============================] - 5s 124ms/step - loss: 1.8390 - accuracy: 0.2974 - val_loss: 1.7780 - val_accuracy: 0.3425
Epoch 7/30
43/43 [==============================] - 5s 124ms/step - loss: 1.8554 - accuracy: 0.3185 - val_loss: 1.7912 - val_accuracy: 0.3185
Epoch

In [20]:
save_loss_plot("loss_nonlinearity_simclr_eval.png", history)
save_acc_plot("acc_nonlinearity_simclr_eval.png", history)